# Google Colab Rust Setup

The following cell is used to set up and spin up a Jupyter Notebook environment with a Rust kernel using Nix and IPC Proxy. 

In [ ]:
!wget -qO- https://gist.github.com/wiseaidev/2af6bef753d48565d11bcd478728c979/archive/3f6df40db09f3517ade41997b541b81f0976c12e.tar.gz | tar xvz --strip-components=1
!bash setup_evcxr_kernel.sh

## Command Line Programs

### Getting Started with clap

In [6]:
use std::process::{Command, Output, Stdio};

// A helper function to execute a shell command from a Rust script
fn execute_command(command: &str) -> Result<(), std::io::Error> {
    let status = Command::new("bash")
        .arg("-c")
        .arg(command)
        .stderr(Stdio::inherit())
        .status()?;

    if status.success() {
        Ok(())
    } else {
        Err(std::io::Error::from_raw_os_error(status.code().unwrap_or(1)))
    }
}

In [2]:
:dep clap = {version = "4.4.11", features = ["derive"]}

In [7]:
use clap::Parser;

#[derive(Parser, Debug)]
#[command(
    author = "Mahmoud Harmouch",
    version = "1.0",
    about = "A command-line find and replace utility",
    name = "find-replace"
)]
struct Args {
    /// Sets the input file to process
    #[arg(short = 'i', long = "input")]
    input: String,
}

fn main() {
    // Access and use the `input` argument here
    let args = Args::parse();

    println!("Parsed Input file name: {}!", args.input);
}

In [8]:
let command = "cd 1-find-replace && cargo run -- --input file.txt";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

    Finished dev [unoptimized + debuginfo] target(s) in 0.02s
     Running `target/debug/find-replace --input file.txt`


Parsed Input file name: file.txt!


()

### Handling Multiple Arguments

In [9]:
use clap::Parser;

#[derive(Parser, Debug)]
#[command(
    author = "Mahmoud Harmouch",
    version = "1.0",
    about = "A command-line find and replace utility",
    name = "Find and Replace"
)]
struct Args {
    /// Sets the input file to process
    #[arg(short = 'i', long = "input")]
    input: String,
    /// Sets the pattern to find
    #[arg(short = 'f', long = "find")]
    find: String,
    /// Sets the replacement text
    #[arg(short = 'r', long = "replace")]
    replace: String,
}

fn main() {
    // Access and use the `input`, `find`, and `replace` arguments here
    let args = Args::parse();
}

In [13]:
let command = "cd 2-multiple-args && cargo run -- -h";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

    Finished dev [unoptimized + debuginfo] target(s) in 0.02s
     Running `target/debug/multiple-args -h`


A command-line find and replace utility

Usage: multiple-args --input <INPUT> --find <FIND> --replace <REPLACE>

Options:
  -i, --input <INPUT>      Sets the input file to process
  -f, --find <FIND>        Sets the pattern to find
  -r, --replace <REPLACE>  Sets the replacement text
  -h, --help               Print help
  -V, --version            Print version


()

### Handling Flag Arguments

In [14]:
use clap::Parser;

#[derive(Parser, Debug)]
#[command(
    author = "Mahmoud Harmouch",
    version = "1.0",
    about = "A command-line find and replace utility",
    name = "Find and Replace"
)]
struct Args {
    /// Sets the input file to process
    #[arg(short = 'i', long = "input")]
    input: String,
    /// Sets the pattern to find
    #[arg(short = 'f', long = "find")]
    find: String,
    /// Sets the replacement text
    #[arg(short = 'r', long = "replace")]
    replace: String,
    /// Perform a case-insensitive search and replace
    #[arg(short = 'c', long = "ignore-case")]
    ignore_case: bool,
}

fn main() {
    // Access and use the `input`, `find`, `replace`, and `ignore-case` arguments here
    let args = Args::parse();
}

In [16]:
let command = "cd 3-flag-args && cargo run -- -h";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

    Finished dev [unoptimized + debuginfo] target(s) in 0.02s
     Running `target/debug/flag-args -h`


A command-line find and replace utility

Usage: flag-args [OPTIONS] --input <INPUT> --find <FIND> --replace <REPLACE>

Options:
  -i, --input <INPUT>      Sets the input file to process
  -f, --find <FIND>        Sets the pattern to find
  -r, --replace <REPLACE>  Sets the replacement text
  -c, --ignore-case        Perform a case-insensitive search and replace
  -h, --help               Print help
  -V, --version            Print version


()

### Advanced Argument Handling

In [17]:
use clap::Args;
use clap::{Parser, Subcommand};

#[derive(Parser, Debug)]
#[command(
    author = "Mahmoud Harmouch",
    version = "1.0",
    about = "A command-line text manipulation utility",
    name = "Text Manipulation Utility"
)]
pub struct Cli {
    /// Turn debugging information on.
    #[arg(short, long, action = clap::ArgAction::Count)]
    debug: u8,
    /// Find and Replace commands.
    #[command(subcommand)]
    pub command: Option<Commands>
}

#[derive(Subcommand, Debug)]
pub enum Commands {
    /// Subcommand for handling find operations.
    Find(FindCommands),
    /// Subcommand for handling replace operations.
    Replace(ReplaceCommands),
}

/// Represents find-related commands.
#[derive(Debug, Args)]
pub struct FindCommands {
    /// Sets the input file to process
    #[clap(short = 'i', long = "input")]
    pub input: Option<String>,
    /// Sets the pattern to find.
    #[clap(short = 'p', long = "pattern")]
    pub pattern: Option<String>,
}

/// Represents repalce-related commands.
#[derive(Debug, Args)]
pub struct ReplaceCommands {
    /// Sets the input file to process
    #[clap(short = 'i', long = "input")]
    pub input: Option<String>,
    /// Sets the pattern to find.
    #[clap(short = 'p', long = "pattern")]
    pub pattern: Option<String>,
    /// Sets the replacement text.
    #[clap(short = 'r', long = "replace")]
    pub replace: Option<String>,
}

fn main() {
    let args = Cli::parse();
    match args.command {
        Some(Commands::Find(command)) => {
            match command.input {
                Some(ref input) => {
                    // cargo run -- find --input file_name
                    println!("{:?}", input);
                }
                None => {
                    println!("Please provide a file name.");
                }
            };
            match command.pattern {
                Some(ref pattern) => {
                    // cargo run -- find --pattern custom_pattern
                    println!("{:?}", pattern);
                }
                None => {
                    println!("Please provide a pattern.");
                }
            };
        }
        Some(Commands::Replace(command)) => {
            match command.input {
                Some(ref input) => {
                    // cargo run -- replace --input file_name
                    println!("{:?}", input);
                }
                None => {
                    println!("Please provide a file name.");
                }
            };
            match command.pattern {
                Some(ref pattern) => {
                    // cargo run -- replace --pattern custom_pattern
                    println!("{:?}", pattern);
                }
                None => {
                    println!("Please provide a pattern.");
                }
            };
            match command.replace {
                Some(ref replace) => {
                    // cargo run -- replace --replace string
                    println!("{:?}", replace);
                }
                None => {
                    println!("Please provide a pattern.");
                }
            };
        }
        None => println!(
            "Unknown command. Use '--help' for usage instructions."
        )
    };
}

In [19]:
let command = "cd 4-advanced-args && cargo run -- -h";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

    Finished dev [unoptimized + debuginfo] target(s) in 0.02s
     Running `target/debug/advanced-args -h`


A command-line text manipulation utility

Usage: advanced-args [OPTIONS] [COMMAND]

Commands:
  find     Subcommand for handling find operations
  replace  Subcommand for handling replace operations
  help     Print this message or the help of the given subcommand(s)

Options:
  -d, --debug...  Turn debugging information on
  -h, --help      Print help
  -V, --version   Print version


()

In [20]:
let command = "cd 4-advanced-args && cargo run -- find -h";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

    Finished dev [unoptimized + debuginfo] target(s) in 0.02s
     Running `target/debug/advanced-args find -h`


Subcommand for handling find operations

Usage: advanced-args find [OPTIONS]

Options:
  -i, --input <INPUT>      Sets the input file to process
  -p, --pattern <PATTERN>  Sets the pattern to find
  -h, --help               Print help


()

In [21]:
let command = "cd 4-advanced-args && cargo run -- replace -h";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

    Finished dev [unoptimized + debuginfo] target(s) in 0.02s
     Running `target/debug/advanced-args replace -h`


Subcommand for handling replace operations

Usage: advanced-args replace [OPTIONS]

Options:
  -i, --input <INPUT>      Sets the input file to process
  -p, --pattern <PATTERN>  Sets the pattern to find
  -r, --replace <REPLACE>  Sets the replacement text
  -h, --help               Print help


()

### Implementing Text Search and Replace Logic

#### Reading the Input File

In [23]:
use clap::Args;
use clap::{Parser, Subcommand};
use std::fs::File;
use std::io::{self, Read};

#[derive(Parser, Debug)]
#[command(
    author = "Mahmoud Harmouch",
    version = "1.0",
    about = "A command-line text manipulation utility",
    name = "Text Manipulation Utility"
)]
pub struct Cli {
    /// Turn debugging information on.
    #[arg(short, long, action = clap::ArgAction::Count)]
    debug: u8,
    /// Find and Replace commands.
    #[command(subcommand)]
    pub command: Option<Commands>
}

#[derive(Subcommand, Debug)]
pub enum Commands {
    /// Subcommand for handling find operations.
    Find(FindCommands),
    /// Subcommand for handling replace operations.
    Replace(ReplaceCommands),
}

/// Represents find-related commands.
#[derive(Debug, Args)]
pub struct FindCommands {
    /// Sets the input file to process
    #[clap(short = 'i', long = "input")]
    pub input: Option<String>,
    /// Sets the pattern to find.
    #[clap(short = 'p', long = "pattern")]
    pub pattern: Option<String>,
}

/// Represents repalce-related commands.
#[derive(Debug, Args)]
pub struct ReplaceCommands {
    /// Sets the input file to process
    #[clap(short = 'i', long = "input")]
    pub input: Option<String>,
    /// Sets the pattern to find.
    #[clap(short = 'p', long = "pattern")]
    pub pattern: Option<String>,
    /// Sets the replacement text.
    #[clap(short = 'r', long = "replace")]
    pub replace: Option<String>,
}

fn read_file_to_string(file_path: &str) -> io::Result<String> {
    let mut file = File::open(file_path)?;
    let mut contents = String::new();
    file.read_to_string(&mut contents)?;

    Ok(contents)
}

fn main() {
    let args = Cli::parse();
    let input_content;
    match args.command {
        Some(Commands::Find(command)) => {
            match command.input {
                Some(ref input) => {
                    // cargo run -- find --input file.txt
                    input_content = read_file_to_string(input).unwrap();
                    println!("Input file content: {:?}", input_content);
                }
                None => {
                    println!("Please provide a file name.");
                }
            };
            match command.pattern {
                Some(ref pattern) => {
                    // cargo run -- find --pattern custom_pattern
                    println!("{:?}", pattern);
                }
                None => {
                    println!("Please provide a pattern.");
                }
            };
        }
        Some(Commands::Replace(command)) => {
            match command.input {
                Some(ref input) => {
                    // cargo run -- replace --input file_name
                    println!("{:?}", input);
                }
                None => {
                    println!("Please provide a file name.");
                }
            };
            match command.pattern {
                Some(ref pattern) => {
                    // cargo run -- replace --pattern custom_pattern
                    println!("{:?}", pattern);
                }
                None => {
                    println!("Please provide a pattern.");
                }
            };
            match command.replace {
                Some(ref replace) => {
                    // cargo run -- replace --replace string
                    println!("{:?}", replace);
                }
                None => {
                    println!("Please provide a pattern.");
                }
            };
        }
        None => println!(
            "Unknown command. Use '--help' for usage instructions."
        )
    };
}

In [27]:
let command = "cd 5-reading-files && cargo run -- find --input file.txt -p pattern";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

    Finished dev [unoptimized + debuginfo] target(s) in 0.02s
     Running `target/debug/reading-files find --input file.txt -p pattern`


Input file content: "Hello World\n"
"pattern"


()

### Find and Replace Logic

In [28]:
use clap::Args;
use clap::{Parser, Subcommand};
use std::fs::File;
use std::io::{self, Read};

#[derive(Parser, Debug)]
#[command(
    author = "Mahmoud Harmouch",
    version = "1.0",
    about = "A command-line text manipulation utility",
    name = "Text Manipulation Utility"
)]
pub struct Cli {
    /// Turn debugging information on.
    #[arg(short, long, action = clap::ArgAction::Count)]
    debug: u8,
    /// Find and Replace commands.
    #[command(subcommand)]
    pub command: Option<Commands>
}

#[derive(Subcommand, Debug)]
pub enum Commands {
    /// Subcommand for handling find operations.
    Find(FindCommands),
    /// Subcommand for handling replace operations.
    Replace(ReplaceCommands),
}

/// Represents find-related commands.
#[derive(Debug, Args)]
pub struct FindCommands {
    /// Sets the input file to process
    #[clap(short = 'i', long = "input")]
    pub input: Option<String>,
    /// Sets the pattern to find.
    #[clap(short = 'p', long = "pattern")]
    pub pattern: Option<String>,
}

/// Represents repalce-related commands.
#[derive(Debug, Args)]
pub struct ReplaceCommands {
    /// Sets the input file to process
    #[clap(short = 'i', long = "input")]
    pub input: Option<String>,
    /// Sets the pattern to find.
    #[clap(short = 'p', long = "pattern")]
    pub pattern: Option<String>,
    /// Sets the replacement text.
    #[clap(short = 'r', long = "replace")]
    pub replace: Option<String>,
}

fn read_file_to_string(file_path: &str) -> io::Result<String> {
    let mut file = File::open(file_path)?;
    let mut contents = String::new();
    file.read_to_string(&mut contents)?;

    Ok(contents)
}

fn find_and_replace(content: &mut String, pattern: &str, replacement: &str) {
    *content = content.replace(pattern, replacement);
}

fn main() {
    let args = Cli::parse();
    let mut input_content = "".to_string();
    let mut pattern_to_find = "".to_string();
    let mut replacement_text = "".to_string();

    match args.command {
        Some(Commands::Find(command)) => {
            if let Some(input) = &command.input {
                input_content = read_file_to_string(input).unwrap();
                println!("Input file content: {:?}", input_content);
            } else {
                println!("Please provide a file name.");
            }
            if let Some(pattern) = &command.pattern {
                pattern_to_find = pattern.clone();
                println!("Pattern: {:?}", pattern);
            } else {
                println!("Please provide a pattern.");
            }
        }
        Some(Commands::Replace(command)) => {
            if let Some(input) = &command.input {
                input_content = read_file_to_string(input).unwrap();
                println!("Input file content: {:?}", input_content);
            } else {
                println!("Please provide a file name.");
            }
            if let Some(pattern) = &command.pattern {
                pattern_to_find = pattern.clone();
                println!("Pattern: {:?}", pattern_to_find);
            } else {
                println!("Please provide a pattern.");
            }
            if let Some(replace) = &command.replace {
                replacement_text = replace.clone();
                println!("Replacement text: {:?}", replacement_text);
            } else {
                println!("Please provide a replacement text.");
            }
        }
        None => println!("Unknown command. Use '--help' for usage instructions.")
    };

    println!("Content Before: {:?}", input_content);
    find_and_replace(&mut input_content, &pattern_to_find, &replacement_text);
    println!("Content After: {:?}", input_content);
}

In [31]:
let command = "cd 6-find-replace-logic && cargo run -- replace -i file.txt -p World -r Rust";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

    Finished dev [unoptimized + debuginfo] target(s) in 0.02s
     Running `target/debug/find-replace-logic replace -i file.txt -p World -r Rust`


Input file content: "Hello World\n"
Pattern: "World"
Replacement text: "Rust"
Content Before: "Hello World\n"
Content After: "Hello Rust\n"


()

### Writing the Modified Content

In [32]:
use clap::Args;
use clap::{Parser, Subcommand};
use std::fs::File;
use std::io::{self, Read, Write};

#[derive(Parser, Debug)]
#[command(
    author = "Mahmoud Harmouch",
    version = "1.0",
    about = "A command-line text manipulation utility",
    name = "Text Manipulation Utility"
)]
pub struct Cli {
    /// Turn debugging information on.
    #[arg(short, long, action = clap::ArgAction::Count)]
    debug: u8,
    /// Find and Replace commands.
    #[command(subcommand)]
    pub command: Option<Commands>
}

#[derive(Subcommand, Debug)]
pub enum Commands {
    /// Subcommand for handling find operations.
    Find(FindCommands),
    /// Subcommand for handling replace operations.
    Replace(ReplaceCommands),
}

/// Represents find-related commands.
#[derive(Debug, Args)]
pub struct FindCommands {
    /// Sets the input file to process
    #[clap(short = 'i', long = "input")]
    pub input: Option<String>,
    /// Sets the pattern to find.
    #[clap(short = 'p', long = "pattern")]
    pub pattern: Option<String>,
}

/// Represents repalce-related commands.
#[derive(Debug, Args)]
pub struct ReplaceCommands {
    /// Sets the input file to process
    #[clap(short = 'i', long = "input")]
    pub input: Option<String>,
    /// Sets the pattern to find.
    #[clap(short = 'p', long = "pattern")]
    pub pattern: Option<String>,
    /// Sets the replacement text.
    #[clap(short = 'r', long = "replace")]
    pub replace: Option<String>,
}

fn read_file_to_string(file_path: &str) -> io::Result<String> {
    let mut file = File::open(file_path)?;
    let mut contents = String::new();
    file.read_to_string(&mut contents)?;

    Ok(contents)
}

fn find_and_replace(content: &mut String, pattern: &str, replacement: &str) {
    *content = content.replace(pattern, replacement);
}

fn write_string_to_file(file_path: &str, content: &str) -> io::Result<()> {
    let mut file = File::create(file_path)?;
    file.write_all(content.as_bytes())?;

    Ok(())
}

fn main() -> io::Result<()> {
    let args = Cli::parse();
    let mut input_content = "".to_string();
    let mut pattern_to_find = "".to_string();
    let mut replacement_text = "".to_string();
    let mut input_file_path = "".to_string();

    match args.command {
        Some(Commands::Find(command)) => {
            if let Some(input) = &command.input {
                input_content = read_file_to_string(input)?;
                println!("Input file content: {:?}", input_content);
            } else {
                println!("Please provide a file name.");
            }
            if let Some(pattern) = &command.pattern {
                pattern_to_find = pattern.clone();
                println!("Pattern: {:?}", pattern);
            } else {
                println!("Please provide a pattern.");
            }
        }
        Some(Commands::Replace(command)) => {
            if let Some(input) = &command.input {
                input_file_path = input.to_string();
                input_content = read_file_to_string(input)?;
                println!("Input file content: {:?}", input_content);
            } else {
                println!("Please provide a file name.");
            }
            if let Some(pattern) = &command.pattern {
                pattern_to_find = pattern.clone();
                println!("Pattern: {:?}", pattern_to_find);
            } else {
                println!("Please provide a pattern.");
            }
            if let Some(replace) = &command.replace {
                replacement_text = replace.clone();
                println!("Replacement text: {:?}", replacement_text);
            } else {
                println!("Please provide a replacement text.");
            }
        }
        None => println!("Unknown command. Use '--help' for usage instructions.")
    };

    println!("File Content Before: {:?}", input_content);
    find_and_replace(&mut input_content, &pattern_to_find, &replacement_text);

    write_string_to_file(&input_file_path, &input_content)?;

    input_content = read_file_to_string(&input_file_path)?;
    println!("File Content After: {:?}", input_content);
    Ok(())
}

In [37]:
let command = "cd 7-writing-to-file && cargo run -- replace -i file.txt -p World -r Rust";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

    Finished dev [unoptimized + debuginfo] target(s) in 0.02s
     Running `target/debug/writing-to-file replace -i file.txt -p World -r Rust`


Input file content: "Hello Rust\n"
Pattern: "World"
Replacement text: "Rust"
File Content Before: "Hello Rust\n"
File Content After: "Hello Rust\n"


()

In [39]:
:dep regex = {version = "1.10.2"} 

### Case-Insensitive Search (Optional)

In [59]:
use clap::Args;
use clap::{Parser, Subcommand};
use std::fs::File;
use std::io::{self, Read, Write};
use regex::Regex;

#[derive(Parser, Debug)]
#[command(
    author = "Mahmoud Harmouch",
    version = "1.0",
    about = "A command-line text manipulation utility",
    name = "Text Manipulation Utility"
)]
pub struct Cli {
    /// Turn debugging information on.
    #[arg(short, long, action = clap::ArgAction::Count)]
    debug: u8,
    /// Find and Replace commands.
    #[command(subcommand)]
    pub command: Option<Commands>
}

#[derive(Subcommand, Debug)]
pub enum Commands {
    /// Subcommand for handling find operations.
    Find(FindCommands),
    /// Subcommand for handling replace operations.
    Replace(ReplaceCommands),
}

/// Represents find-related commands.
#[derive(Debug, Args)]
pub struct FindCommands {
    /// Sets the input file to process
    #[clap(short = 'i', long = "input")]
    pub input: Option<String>,
    /// Sets the pattern to find.
    #[clap(short = 'p', long = "pattern")]
    pub pattern: Option<String>,
}

/// Represents repalce-related commands.
#[derive(Debug, Args)]
pub struct ReplaceCommands {
    /// Sets the input file to process
    #[clap(short = 'i', long = "input")]
    pub input: Option<String>,
    /// Sets the pattern to find.
    #[clap(short = 'p', long = "pattern")]
    pub pattern: Option<String>,
    /// Sets the replacement text.
    #[clap(short = 'r', long = "replace")]
    pub replace: Option<String>,
    /// Perform a case-insensitive search and replace.
    #[clap(short = 'c', long = "ignore-case")]
    pub ignore_case: bool,

}

fn read_file_to_string(file_path: &str) -> io::Result<String> {
    let mut file = File::open(file_path)?;
    let mut contents = String::new();
    file.read_to_string(&mut contents)?;

    Ok(contents)
}

fn find_and_replace(content: &mut String, pattern: &str, replacement: &str, ignore_case: bool) {
    let regex = if ignore_case {
        Regex::new(&format!(r"(?i){}", pattern)).unwrap()
    } else {
        Regex::new(pattern).unwrap()
    };

    *content = regex.replace_all(content, replacement).to_string();
}

fn write_string_to_file(file_path: &str, content: &str) -> io::Result<()> {
    let mut file = File::create(file_path)?;
    file.write_all(content.as_bytes())?;

    Ok(())
}

fn main() -> io::Result<()> {
    let args = Cli::parse();
    let mut input_content = "".to_string();
    let mut _pattern_to_find = "".to_string();
    let mut replacement_text = "".to_string();
    let mut input_file_path = "".to_string();
    let mut _ignore_case = false;

    match args.command {
        Some(Commands::Find(command)) => {
            if let Some(input) = &command.input {
                input_content = read_file_to_string(input)?;
                println!("Input file content: {:?}", input_content);
            } else {
                println!("Please provide a file name.");
            }
            if let Some(pattern) = &command.pattern {
                _pattern_to_find = pattern.clone();
                println!("Pattern: {:?}", pattern);
            } else {
                println!("Please provide a pattern.");
            }
        }
        Some(Commands::Replace(command)) => {
            if let Some(input) = &command.input {
                input_file_path = input.to_string();
                input_content = read_file_to_string(input)?;
                println!("Input file content: {:?}", input_content);
            } else {
                println!("Please provide a file name.");
            }
            if let Some(pattern) = &command.pattern {
                _pattern_to_find = pattern.clone();
                println!("Pattern: {:?}", _pattern_to_find);
            } else {
                println!("Please provide a pattern.");
            }
            if let Some(replace) = &command.replace {
                replacement_text = replace.clone();
                println!("Replacement text: {:?}", replacement_text);
            } else {
                println!("Please provide a replacement text.");
            }
            _ignore_case = command.ignore_case;
            println!("Ignore case: {:?}", _ignore_case);
            println!("File Content Before: {:?}", input_content);
            find_and_replace(&mut input_content, &_pattern_to_find, &replacement_text, _ignore_case);

            write_string_to_file(&input_file_path, &input_content)?;

            input_content = read_file_to_string(&input_file_path)?;
            println!("File Content After: {:?}", input_content);
        }
        None => println!("Unknown command. Use '--help' for usage instructions.")
    };

    Ok(())

}

In [58]:
let command = "cd 8-case-insensitive-search && cargo run -- replace -i file.txt -p world -r Rust -c";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

    Finished dev [unoptimized + debuginfo] target(s) in 0.02s
     Running `target/debug/case-insensitive-search replace -i file.txt -p world -r Rust -c`


Input file content: "Hello World\n"
Pattern: "world"
Replacement text: "Rust"
Ignore case: true
File Content Before: "Hello World\n"
File Content After: "Hello Rust\n"


()

### Multiple Input Files

In [62]:
use clap::Args;
use clap::{Parser, Subcommand};
use std::fs::File;
use std::io::{self, Read, Write};
use regex::Regex;

#[derive(Parser, Debug)]
#[command(
    author = "Mahmoud Harmouch",
    version = "1.0",
    about = "A command-line text manipulation utility",
    name = "Text Manipulation Utility"
)]
pub struct Cli {
    /// Turn debugging information on.
    #[arg(short, long, action = clap::ArgAction::Count)]
    debug: u8,
    /// Find and Replace commands.
    #[command(subcommand)]
    pub command: Option<Commands>
}

#[derive(Subcommand, Debug)]
pub enum Commands {
    /// Subcommand for handling find operations.
    Find(FindCommands),
    /// Subcommand for handling replace operations.
    Replace(ReplaceCommands),
}

/// Represents find-related commands.
#[derive(Debug, Args)]
pub struct FindCommands {
    /// Sets the input file to process
    #[clap(short = 'i', long = "input", num_args = 1.., value_delimiter = ' ')]
    pub input: Option<Vec<String>>,
    /// Sets the pattern to find.
    #[clap(short = 'p', long = "pattern")]
    pub pattern: Option<String>,
}

/// Represents repalce-related commands.
#[derive(Debug, Args)]
pub struct ReplaceCommands {
    /// Sets the input file to process
    #[clap(short = 'i', long = "input", num_args = 1.., value_delimiter = ' ')]
    pub input: Option<Vec<String>>,
    /// Sets the pattern to find.
    #[clap(short = 'p', long = "pattern")]
    pub pattern: Option<String>,
    /// Sets the replacement text.
    #[clap(short = 'r', long = "replace")]
    pub replace: Option<String>,
    /// Perform a case-insensitive search and replace.
    #[clap(short = 'c', long = "ignore-case")]
    pub ignore_case: bool,

}

fn read_file_to_string(file_path: &str) -> io::Result<String> {
    let mut file = File::open(file_path)?;
    let mut contents = String::new();
    file.read_to_string(&mut contents)?;

    Ok(contents)
}

fn find_and_replace(content: &mut String, pattern: &str, replacement: &str, ignore_case: bool) {
    let regex = if ignore_case {
        Regex::new(&format!(r"(?i){}", pattern)).unwrap()
    } else {
        Regex::new(pattern).unwrap()
    };

    *content = regex.replace_all(content, replacement).to_string();
}

fn write_string_to_file(file_path: &str, content: &str) -> io::Result<()> {
    let mut file = File::create(file_path)?;
    file.write_all(content.as_bytes())?;

    Ok(())
}

fn main() -> io::Result<()> {
    let args = Cli::parse();
    let mut input_files: Vec<String> = vec!["".to_string(),];
    let mut pattern_to_find = "".to_string();
    let mut replacement_text = "".to_string();
    let mut ignore_case = false;

    match args.command {
        Some(Commands::Find(command)) => {
            if let Some(input) = &command.input {
                input_files = input.to_vec();
            } else {
                println!("Please provide a file name.");
            }
            if let Some(pattern) = &command.pattern {
                pattern_to_find = pattern.clone();
                println!("Pattern: {:?}", pattern);
            } else {
                println!("Please provide a pattern.");
            }

            for input_file_path in input_files {
                let mut input_content = read_file_to_string(&input_file_path)?;

                find_and_replace(&mut input_content, &pattern_to_find, "", ignore_case);

                println!("Found Content: {:?}", input_content);
            }
        }
        Some(Commands::Replace(command)) => {
            if let Some(input) = &command.input {
                input_files = input.to_vec();
            } else {
                println!("Please provide a file name.");
            }
            if let Some(pattern) = &command.pattern {
                pattern_to_find = pattern.clone();
                println!("Pattern: {:?}", pattern_to_find);
            } else {
                println!("Please provide a pattern.");
            }
            if let Some(replace) = &command.replace {
                replacement_text = replace.clone();
                println!("Replacement text: {:?}", replacement_text);
            } else {
                println!("Please provide a replacement text.");
            }
            if let ignore = &command.ignore_case {
                ignore_case = *ignore;
                println!("Ignore case: {:?}", ignore_case);
            } else {
                println!("Please provide if case sensitive replacement.");
            }

            for input_file_path in input_files {
                let mut input_content = read_file_to_string(&input_file_path)?;

                println!("File Content Before: {:?}", input_content);
                find_and_replace(&mut input_content, &pattern_to_find, &replacement_text, ignore_case);

                // Write the modified content back to the input file
                write_string_to_file(&input_file_path, &input_content)?;

                input_content = read_file_to_string(&input_file_path)?;
                println!("File Content After: {:?}", input_content);
            }

        }
        None => println!("Unknown command. Use '--help' for usage instructions.")
    };

    Ok(())

}

In [69]:
let command = "cd 9-multiple-input-files && cargo run -- replace -i file1.txt file2.txt file3.txt -p world -r Rust -c";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

    Finished dev [unoptimized + debuginfo] target(s) in 0.02s
     Running `target/debug/multiple-input-files replace -i file1.txt file2.txt file3.txt -p world -r Rust -c`


Pattern: "world"
Replacement text: "Rust"
Ignore case: true
File Content Before: "Hello World\n"
File Content After: "Hello Rust\n"
File Content Before: "Hello World\n"
File Content After: "Hello Rust\n"
File Content Before: "Hello World\n"
File Content After: "Hello Rust\n"


()

---
---